In [61]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from  langserve import add_routes



In [62]:
load_dotenv()

qroq_apikey = os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=qroq_apikey)

In [63]:
#
from langchain_core.messages import HumanMessage
human_mesge = [HumanMessage(content="Hi, my name is albin and i am cheif ai engineer")]
model.invoke(human_mesge)

AIMessage(content="Nice to meet you, Chief AI Engineer Albin. It's great to have you here. What brings you to our platform today? Are you working on a new AI project, looking for advice on a specific challenge, or just exploring the latest developments in the field?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 49, 'total_tokens': 104, 'completion_time': 0.102587957, 'prompt_time': 0.002433342, 'queue_time': 0.087961423, 'total_time': 0.105021299}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e29789e0-30dc-417b-aacc-d827840a9f89-0', usage_metadata={'input_tokens': 49, 'output_tokens': 55, 'total_tokens': 104})

In [64]:
from langchain_core.messages import AIMessage

ai_msge = [HumanMessage(content="Hi, my name is albin and i am cheif ai engineer"),
           AIMessage(content="Nice to meet you, Albin. As a Chief Operations and Maintenance (O&M) Engineer, you play a crucial role in ensuring the smooth operation and reliability of facilities, equipment, and processes. Your expertise is essential in maintaining the efficiency and productivity of the organization.\n\nWhat specific challenges or projects are you currently working on, or would you like to discuss about O&M engineering in general? "),
           HumanMessage(content="Hey, what is my name ? what i am do ?"),
           ]


model.invoke(ai_msge)

AIMessage(content='Your name is Albin, and you are a Chief AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 150, 'total_tokens': 165, 'completion_time': 0.022666744, 'prompt_time': 0.008299758, 'queue_time': 0.089754065, 'total_time': 0.030966502}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--4e459b85-703c-4c7c-bba6-56c43fb6052c-0', usage_metadata={'input_tokens': 150, 'output_tokens': 15, 'total_tokens': 165})

# Message History

In [65]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [66]:
#if multoplie pepole chat with the model => how to hnadle it ==> session

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # craete new one if not found if
    return store[session_id] #retrun chat associated with that session id
        
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [67]:
config1 = {"configurable" : {"session_id": "chat1"}}

In [68]:
responce = with_message_history.invoke(
    [HumanMessage(content="My name is albin, and i am learning GenAI")],
    config=config1
    )

responce.content

"Hello Albin, I'm happy to meet you and help you on your journey to learning GenAI (Generative Artificial Intelligence). What specific aspects of GenAI would you like to explore or learn about? Would you like to start with the basics, or dive into something more advanced like transformer architectures, generative adversarial networks (GANs), or language models?"

In [69]:
#change > sessionid

config1 = {"configurable" : {"session_id": "chat2"}}
responce = with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config1
    )

responce.content


"I don't have any information about your name. I'm a large language model, I don't have the ability to retain any personal information about individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!"

In [70]:
#check the chat message histroy => sessionid

config1 = {"configurable" : {"session_id": "chat1"}}
responce = with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config1
    )

responce.content

'Your name is Albin.'

# prompt template

In [71]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [72]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are powerful ai model, Answer all question at your very best..."),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)

chain = prompt | model
result = chain.invoke({"messages" : [HumanMessage(content="My name is Albin")]})
result.content

'Nice to meet you, Albin. How can I assist you today?'

In [73]:
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable" : {"session_id" : "chat3"}}
res  = with_message_history.invoke(
        [HumanMessage(content="My name is Albin")],
        config=config
    )
res.content

'Nice to meet you, Albin. Is there something I can help you with today?'

In [74]:
## add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are powerful ai model, Answer all question at your very best in {language}"),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
)

chain = prompt | model
result = chain.invoke(
    {
        "messages" : [HumanMessage(content="My name is Albin")], 
        "language" : "German"
    }
)
result.content

'Hallo Albin, wie kann ich dir heute helfen?'

In [75]:
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages",history_messages_key="messages")

config = {"configurable": {"session_id": "chat_mueller_1"}}
res2  = with_message_history.invoke(
    {
        "input" : [HumanMessage(content="My name is Albin")], 
        "language" : "German"
    },
        config=config 
    )
res2.content

'Nice to meet you, Albin. Is there something I can help you with or would you like to chat?'

# Manage Chat Histroy

In [76]:
from langchain_core.messages import SystemMessage, trim_messages

# ' trim_messages' = reduce how many message we are sending to the model

In [77]:
trimmer = trim_messages(
    max_tokens=70,          #should be exact ´same params name
    strategy="last",        #should be exact ´same params name
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages1 = [
    SystemMessage(content="You are the good assistant"),
    HumanMessage(content="Hi! I am Müller"),
    AIMessage(content="Hai!!"),
    HumanMessage(content="i Like vanilla ice cream"),
    AIMessage(content="oh that nice"),
    HumanMessage(content="what is 9+2"),
    AIMessage(content="that 11 Müller"),
    HumanMessage(content="thank you"),
    AIMessage(content="NO probelm"),
]

trimmer.invoke(messages1)


[SystemMessage(content='You are the good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! I am Müller', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hai!!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i Like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='oh that nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 9+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='that 11 Müller', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thank you', additional_kwargs={}, response_metadata={}),
 AIMessage(content='NO probelm', additional_kwargs={}, response_metadata={})]

In [78]:
# how to pass trimmer in a chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


trimmed = RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
chain = trimmed | prompt | model

response  = chain.invoke({
    "messages" : messages1 + [HumanMessage(content="what is 9+2 ? ")],
    "language" : "English"
}
)

responce.content

'Your name is Albin.'

In [79]:
# wrpa message hiostory
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="input")

config = {"configurable" : {"session_id" : "chat5"}}
res2  = with_message_history.invoke(
    {
        "input" : [HumanMessage(content="what math question i asked")], 
        "language" : "englsih"
    },
        config=config 
    )
res2.content

"You didn't ask a math question. This is the start of our conversation. If you have a math question, feel free to ask, and I'll do my best to help you."